# Introdução

## Gestão de Bibliotecas

### Instalando depedências

In [ ]:
pip install adal

Python interpreter will be restarted.
Requirement already satisfied: adal in /databricks/python3/lib/python3.8/site-packages (1.2.7)
Requirement already satisfied: PyJWT<3,>=1.0.0 in /databricks/python3/lib/python3.8/site-packages (from adal) (2.4.0)
Requirement already satisfied: python-dateutil<3,>=2.1.0 in /databricks/python3/lib/python3.8/site-packages (from adal) (2.8.1)
Requirement already satisfied: requests<3,>=2.0.0 in /databricks/python3/lib/python3.8/site-packages (from adal) (2.25.1)
Requirement already satisfied: cryptography>=1.1.0 in /databricks/python3/lib/python3.8/site-packages (from adal) (37.0.2)
Requirement already satisfied: cffi>=1.12 in /databricks/python3/lib/python3.8/site-packages (from cryptography>=1.1.0->adal) (1.14.5)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.8/site-packages (from cffi>=1.12->cryptography>=1.1.0->adal) (2.20)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil<3,>=2.1.0->adal) (1.15.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.0.0->adal) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.0.0->adal) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.0.0->adal) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests<3,>=2.0.0->adal) (2.10)
WARNING: You are using pip version 21.0.1; however, version 22.1.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-5c4e56f0-fc7c-48a5-8b76-b0fa68da7e76/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

### Importando dependências

In [ ]:
from pyspark.sql.functions import regexp_replace, regexp_extract, col
import adal

## Descrição de conteúdo das tabelas

**DataFrame de Eventos**

Colunas|Descrição|
-------|---------|
classificacaoEtaria|Idade mínima para ter acesso ao conteúdo. Varia de Livre a +18
id_event|Identificador do evento
name|Nome do evento
rule.duration|Duração do evento
rule.endsAt|Horário de término do evento
rule.frequency|Frequência de ocorrência do evento
rule.price|Preço de participação do evento
rule.spaceId|Identificador de espaço que irá ocorrer o evento
rule.startsAt|Horário de ínicio
rule.startsOn|Data de início do evento
rule.until|Data final do evento
shortDescription|Pequena descrição de conteúdo do evento

**DataFrame de espaços/Museus**

Colunas|Descrição|
-------|---------|
id|Identificador do museu/espaço
location.latitude|Latitude do museu/espaço 
location.longitude|Longitude do museu/espaço 
name|Nome do museu/espaço

## Carregamento de dados

In [ ]:
df_event = spark.read.format("json").option("inferSchema", "true").load("/mnt/storage_name/lab4/event.json")
df_space = spark.read.format("json").option("inferSchema", "true").load("/mnt/storage_name/lab4/space.json")

# Limpeza dos dados

In [ ]:
df_event, df_space = df_event.drop('index'), df_space.drop('index')

## Limpeza DataFrame Evento

In [ ]:
df_event = df_event.withColumnRenamed('rule.duration', 'duration')\
                   .withColumnRenamed('rule.endsAt', 'endsAt')\
                   .withColumnRenamed('rule.frequency', 'frequency')\
                   .withColumnRenamed('rule.spaceId', 'spaceId')\
                   .withColumnRenamed('rule.startsAt', 'startsAt')\
                   .withColumnRenamed('rule.startsOn', 'startsOn')\
                   .withColumnRenamed('rule.price', 'price')\
                   .withColumnRenamed('rule.until', 'until')\
                   .withColumnRenamed('name', 'name_event')

In [ ]:
df_event = df_event.withColumn('price',regexp_replace(col('price'), '(entrada gratuita|Entrada Gratuita)|(Entrada Franca|entrada franca)|(grátis|Grátis|GRÁTIS)|R\$ 00,00|^0', 'Gratuito'))\
                   .withColumn('price',regexp_replace(col('price'), '^(Gratuito)?,[0]{1,3}|^(Gratuito)?[0]{1,3}|GRATUITO|Gratuita|gratuito|Entrada gratuita', 'Gratuito'))\
                   .withColumn('price',regexp_replace(col('price'), '(^[0-9])', 'R\$ $1'))\
                   .withColumn('price',regexp_replace(col('price'), '(^R\$)([0-9])', '$1 $2'))\
                   .withColumn('price',regexp_replace(col('price'), '(Gratuito)(\.)', '$1'))\
                   .withColumn('price',regexp_replace(col('price'), '(livre)', 'Livre'))\
                   .withColumn('price',regexp_replace(col('price'), '(R\$ 6,00\/3,00)...(inteira\/meia)|(Inteira\/meia)...(R\$6,00\/3,00)[$\$]|(Inteira\/meia)...(R\$6,00\/3,00)', '$1$4$6 Inteira/Meia'))\
                   .withColumn('price',regexp_replace(col('price'), '(^R\$)([0-9])', '$1 $2'))\
                   .withColumn('price',regexp_replace(col('price'), '(null)', 'Não informado'))

In [ ]:
df_event = df_event.withColumn('name_event',regexp_replace(col('name_event'), '["]', ''))\
                   .withColumn('name_event',regexp_replace(col('name_event'), '[#•]', ''))\
                   .withColumn('name_event',regexp_replace(col('name_event'), '^[,]', ''))\
                   .withColumn('name_event',regexp_replace(col('name_event'), '(\A[0-9]{2}|\A[0-9]{1})[ª|\s|°|º]+(?!anos|ANOS)', '$1ª '))


In [ ]:
df_event = df_event.withColumn('frequency',regexp_replace(col('frequency'), '(daily)', 'Diário'))\
                   .withColumn('frequency',regexp_replace(col('frequency'), '(weekly)', 'Semanal'))\
                   .withColumn('frequency',regexp_replace(col('frequency'), '(once)', 'Único'))

In [ ]:
df_event = df_event.fillna(value='Não informado',subset=["price"])

In [ ]:
df_space =  df_space.withColumnRenamed('location.latitude', 'latitude')\
            .withColumnRenamed('location.longitude', 'longitude')\
            .withColumnRenamed('id', 'id_space')\
            .withColumnRenamed('name', 'name_space')

root
-- id_space: long (nullable = true)
-- latitude: string (nullable = true)
-- longitude: string (nullable = true)
-- name_space: string (nullable = true)

# Criação e ingestão da tabela stage

In [ ]:
resource_app_id_url = "https://database.windows.net/"

service_principal_id = dbutils.secrets.get(scope = "scope_name", key = "key")
service_principal_secret = dbutils.secrets.get(scope = "scope_name", key = "key")
tenant_id = "*********"
authority = "https://login.windows.net/" + tenant_id

azure_sql_url = "jdbc:sqlserver://sql-server.database.windows.net"
database_name = "db_name"

encrypt = "true"
host_name_in_certificate = "*.database.windows.net"

context = adal.AuthenticationContext(authority)
token = context.acquire_token_with_client_credentials(resource_app_id_url, service_principal_id, service_principal_secret)
access_token = token["accessToken"]

In [ ]:
df_stage = df_space.join(df_event,df_space.id_space ==  df_event.spaceId,"left")

Out[83]: 5136

In [ ]:
db_table = "STAGE_name.DL_IPAV"

df_stage.write \
.format("jdbc") \
.mode("overwrite") \
.option("url", azure_sql_url) \
.option("dbtable", db_table) \
.option("databaseName", database_name) \
.option("accessToken", access_token) \
.option("encrypt", "true") \
.option("hostNameInCertificate", "*.database.windows.net") \
.save()